# _k_-means Clustering mit Scikit-learn

Bei unüberwachten Machine Learning Verfahren geht es darum, die Struktur der Daten zu erforschen und nach unbekannten Mustern zu suchen, um sinnvolle und nützliche Informationen daraus zu extrahieren. Eins der am häufigsten benutzten unüberwachten Datenanalyseverfahren ist _Clustering_, dessen Ziel ist es, ähnliche Beobachtungen in Gruppen bzw. _Cluster_ einzuteilen.

## Vorbereitung des Datensatzes

Der _k-means_ Clustering-Algorithmus gehört zu den sogenannten _partitionierenden_ Verfahren. Das bedeutet, dass er jede Beobachtung eines Datensatzes einem der Cluster zuteilt. Da beim Clustering noch keine Informationen über die Struktur der Daten zur Verfügung stehen – denn diese sollen durch das Clustering herausgefunden werden – kann man nach der Aufteilung des Datensatzes in Cluster erstmal nicht sagen, ob die Zerlegung auf irgendeine Weise sinnvoll oder nützlich ist. Um das Prinzip und mögliche Schwierigkeiten von Clustering zu verdeutlichen, verwenden wir im Folgenden einen Datensatz, bei dem die Beobachtungen bereits bestimmten Klassen zugeordnet wurden. Nach der Aufteilung des Datensatzes mit dem _k-means_ Clustering-Algorithmus können die Zuordnung der Beobachtungen zu den Clustern mit den originalen Klassenlabels verglichen werden, um bewerten zu können, ob die gefundenen Cluster sinnvoll sind.

Zuerst aber müssen alle Module importiert werden, die im Programm gebraucht werden.

In [ ]:
# Importiere die nötigen Module
from sklearn import cluster, preprocessing
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

Im Folgenden arbeiten wir mit dem <a href="https://allisonhorst.github.io/palmerpenguins/">_Palmer Archipelago (Antarctica) Penguin_</a> Datensatz. Dieser Datensatz enthält Informationen über 344 Pinguine, die auf drei Inseln des Palmer-Archipels in der Antarktis beheimatet sind. Die Merkmale bzw. Features des Datensatzes enthalten verschiedene Messdaten, die über die Pinguine gesammelt wurden, wie Länge und Höhe des Schnabels, Länge der Flosse und die Körpermasse bzw. das Gewicht. Außerdem enthält das Merkmal _species_ die Pingiunart der Pinguine. Dabei gehören die Beobachtungen bzw. die Pinguine im Datensatz zu einer der drei Arten, _Adelie_, _Chinstrap_ (deutsch:  Zügelpinguin) und _Gentoo_ (deutsch: Eselspinguin).

In [ ]:
# Lade den Palmer Archipelago (Antarctica) Penguin Datensatz
penguins = sns.load_dataset("penguins")

# Wandle das DataFrame in ein NumPy-Array um
penguins_arr = penguins.to_numpy()

Der Datensatz wird in eine Struktur _DataFrame_ geladen und in ein _NumPy_-Array mit der Funktion <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html">_to_numpy()_</a> umgewandelt. Für die weitere Analyse werden nur die Merkmale _Schnabellänge_, _Schnabelhöhe_, _Flossenlänge_ und _Körpermasse_ in die Datenmatrix und die entsprechende Pinguinart in das Zielarray gespeichert.

In [ ]:
# Speichere das dritte (Schnabellänge), das vierte (Schnabelhöhe),
# das fünfte (Flossenlänge) und das sechste (Körpermasse)
# Merkmal in der Datenmatrix
X_raw = np.array(penguins_arr[:,2:6]).astype(float)

# Speichere das erste Merkmal als Zielmerkmal
y_raw = penguins_arr[:,0]

Da zwei Beobachtungen fehlende Werte enthalten, werden diese aus dem Datensatz entfernt. So verbleiben noch 342 Beobachtungen im Datensatz.

In [ ]:
# Lösche alle Zeilen aus der Datenmatrix, die
# fehlende Wert enthalten
X = X_raw[~np.isnan(X_raw).any(axis=1)]
y = y_raw[~np.isnan(X_raw).any(axis=1)]

Die Beobachtungen werden durch ihre Werte für die Schnabellänge, die Schnabelhöhe und die Flossenlänge in einem 3-dimensionalen Diagramm dargestellt. Die Zugehörigkeit zur Pinguinart wird durch unterschiedliche Farben der Datenpunkte markiert.

In [ ]:
pyo.init_notebook_mode()
fig_orig = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2], color=y)
fig_orig.show()

Es wird davon ausgegangen, dass die Zuordnung der Pinguine zu unterschiedlichen Pinguinarten nicht bekannt ist und diese noch erforscht werden muss. Also werden die Beobachtungen in einem 3-dimensionalen Diagramm grafisch dargestellt, ohne die Klassenzugehörigkeit kenntlich zu machen.

In [ ]:
fig1 = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2])
fig1.show()

## _k-means_ Clustering-Algorithmus

Der _k-means_ Clustering-Algorithmus teilt alle Beobachtungen eines Datensatzes in _k_ Cluster auf. Die Anzahl der Cluster ist in der Regel vorher nicht bekannt. Für den Pinguin-Datensatz wird aber k = 3 angenommen.

Ein _k-means_ Clustering-Modell wird mit der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html">_KMeans()_</a> des _Scikit-learn_-Untermoduls <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster">_Clustering_</a> erstellt.
* Mit dem Parameter _n\_clusters_ wird die Anzahl der Cluster festgelegt, in die die Beobachtungen eines Datensatzes zerlegt werden.
* In der Regel werden die Mittelwerte bzw. Zentren der Cluster am Anfang mit zufällig aus dem Datensatz ausgewählten Beobachtungen initialisiert. In den Voreinstellungen der Funktion _KMeans()_ ist aber eine andere ausgeklügeltere, aber aufwändigere Initialisierungsmethode festgelegt. Durch die Zuweisung des Wertes _random_ für den Parameter _init_ wird sichergestellt, dass die _k_ zufällig aus dem Datensatz ausgewählten Beobachtungen als anfängliche Clustermittelwerte fungieren.
* Der Parameter _random\_state_ bestimmt die entsprechende Zufallszahlengenerierung für die Wahl der anfänglichen Clusterzentren. Um die Reproduzierbarkeit der Clusteringergebnisse zu garantieren, soll diesem Parameter ein Wert zugewiesen werden.
* Mit dem Parameter _n\_init_ wird die Anzahl der Durchläufe des _k-means_ Algorithmus mit jeweils unterschiedlichen zufällig ausgewählten Start-Clusterzentren bestimmt. Das heißt, dass der _k-means_ Algorithmus mehrere Male mit jeweils unterschiedlichen anfänglichen Clusterzentren gestartet wird. Als Ergebnis wird die Aufteilung der Beobachtungen in die Cluster ausgegeben, bei der die Beobachtungen am nächsten an den Zentren ihrer Cluster liegen.

Die Funktion _KMeans()_ hat auch weitere Parameter, mit denen das Clustering-Modell exakter spezifiziert werden kann. Nähere Informationen zu diesen sind in der <a href="https://scikit-learn.org/stable/modules/classes.html">API Refernce</a> von Scikit-learn zu finden.

Mit der Funktion _fit()_ wird die Aufteilung der Beobachtungen des Datensatzes in Cluster mit dem erstellten Clustering-Modell berechnet.

In [ ]:
# Erstelle das Clustering-Modell für k = 3
k_means = cluster.KMeans(n_clusters=3, init="random",
                         random_state=42, n_init=10)

# Berechne das Clustering für den Datensatz X
k_means.fit(X)

Die Aufteilung der Beobachtungen in Cluster wird in einem 3-dimensionalen Diagramm dargestellt. Dafür werden wieder nur die ersten drei Merkmale, die Schnabellänge, die Schnabelhöhe und die Flossenlänge, verwendet. Die Clusterzuweisungen der Beobachtungen werden durch unterschiedliche Farben markiert. Dabei werden die Labels bzw. die Clusterzuweisungen der Beobachtungen mit der Anweisung _labels\__ ausgegeben. Da die Clusterlabels mit ganzen Zahlen angegeben werden, werden diese in Zeichenketten umgewandelt, damit sie als kategoriale Merkmalswerte von der _scatter()_-Funktion wahrgenommen werden.

In [ ]:
# Plotte das Clusteringergebnis
fig2 = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2],
                     color=k_means.labels_.astype(str))
fig2.show()

Die Beobachtungen werden anhand der Merkmale Schnabellänge, Schnabelhöhe und Gewicht dargestellt.

In [ ]:
# Plotte das Clusteringergebnis
fig3 = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,3],
                     color=k_means.labels_.astype(str))
fig3.show()

Die Spannweiten der Werte einzelner Merkmale bzw. Features des Datensatzes werden berechnet.

In [ ]:
# Gebe die Spannweiten der Werte einzelner
# Merkmale des Datensatzes aus
print(np.max(X, axis=0)-np.min(X, axis=0))

Um allen Merkmalen die gleiche Bedeutung bei der Distanzberechnung zuzuweisen, sollten die Werte aller Merkmale vor dem Clustering am besten normalisiert werden. Bei der _Normalisierung_ handelt es sich um eine Methode, bei der die Werte numerischer Merkmale bzw. Features so geändert werden, dass sie die gleiche Größenordnung haben. Dabei sollten die Unterschiede in den Wertebereichen nicht verzerrt werden. Meistens werden die Merkmalswerte auf das Intervall [0, 1] normalisiert.

Das _Scikit-learn_-Untermodul <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing">_Preprocessing and Normalization_</a> bietet eine Funktion namens <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html">_minmax_scale()_</a> an, mit deren Hilfe die Werte jedes Merkmals im Datensatz auf einen vorgegebenen Bereich normalisiert werden können. In der Voreinstellung der Funktion ist der Wertebereich [0, 1] festgelegt.

In [ ]:
# Normalisiere die Merkmalswerte auf den Bereich [0,1]
X_normalized = preprocessing.minmax_scale(X)

Die Spannweiten der Werte einzelner Merkmale bzw. Features des normalisierten Datensatzes werden berechnet.

In [ ]:
# Gebe die Spannweite der normalisierten Werte einzelner
# Merkmale des Datensatzes aus
print(np.max(X_normalized, axis=0)-np.min(X_normalized, axis=0))

Die Aufteilung der Beobachtungen des normalisierten Datensatzes in Cluster wird mit dem oben erstellten Clustering-Modell berechnet.

In [ ]:
# Berechne das Clustering für
# den Datensatz X_normalized
k_means.fit(X_normalized)

Die Aufteilung der Beobachtungen in Cluster wird in einem 3-dimensionalen Diagramm dargestellt. Dafür werden die ersten drei Merkmale, die Schnabellänge, die Schnabelhöhe und die Flossenlänge verwendet. Die Clusterzuweisungen der Beobachtungen werden durch unterschiedliche Farben markiert. Dabei werden die tatsächlichen Klassen der Beobachtungen durch Kreise und die von _k-means_ Algorithmus berechneten Clusterzuweisungen durch Kreuze dargestellt.

In [ ]:
# Visualisiere die tatsächliche Klassenzugehörigkeit der
# Beobachtungen als Kreis und die von k-means berechnete
# Clusterzugehörigkeit als Kreuz
fig4 = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2],
                     color=y, opacity=0.8,
                     symbol_sequence=["circle"])
fig5 = px.scatter_3d(x=X[:, 0], y=X[:, 1], z=X[:, 2],
                     color=k_means.labels_.astype(str),
                     symbol_sequence=["cross"])
fig6 = go.Figure(data=fig4.data + fig5.data)
fig6.show()